In [1]:
from shapely.geometry import Point
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster, HeatMap, HeatMapWithTime
import ipywidgets as widgets
from IPython.display import HTML, display
from collections import defaultdict

In [2]:
eq_table = pd.read_csv('../Data/04_deadly_eq_mag_cleaned.csv', index_col=False)

In [3]:
eq_table.columns

Index(['origin (utc)', 'present-day country and link to wikipedia article',
       'lat', 'long', 'depth (km)', 'magnitude', 'secondary effects',
       'pde shaking deaths', 'pde total deaths', 'utsu total deaths',
       'em-dat total deaths', 'other source deaths', 'deaths',
       'original_magnitude', 'original_magnitude_scale'],
      dtype='object')

In [4]:
eq_table['origin (utc)'] = pd.to_datetime(eq_table['origin (utc)'])
eq_table['origin_utc_string'] = eq_table['origin (utc)'].dt.strftime("%Y-%m-%dT%H:%M:%S")

In [5]:
eq_table['lat'] = pd.to_numeric(eq_table['lat'], errors='coerce')

In [6]:
eq_table['long'] = pd.to_numeric(eq_table['long'], errors='coerce')

In [7]:
def lat_long_to_point(row):
    return Point(row['long'], row['lat'])

In [8]:
eq_table['position'] = eq_table.apply(lat_long_to_point, axis=1)

In [9]:
eq_table_geo = gpd.GeoDataFrame(eq_table, crs='epsg:4326', geometry=eq_table['position'])

In [10]:
eq_table_geo.columns

Index(['origin (utc)', 'present-day country and link to wikipedia article',
       'lat', 'long', 'depth (km)', 'magnitude', 'secondary effects',
       'pde shaking deaths', 'pde total deaths', 'utsu total deaths',
       'em-dat total deaths', 'other source deaths', 'deaths',
       'original_magnitude', 'original_magnitude_scale', 'origin_utc_string',
       'position', 'geometry'],
      dtype='object')

In [11]:
eq_table_geo['original_magnitude'] = eq_table_geo['original_magnitude'].fillna(0.0)

In [12]:
world_map = folium.Map(tiles='Stamen Terrain')

In [13]:
heatmapdata = [
        (row['geometry'].y, row['geometry'].x, row['original_magnitude']) for idx, row 
            in eq_table_geo.iterrows() 
                if row['geometry'] and not pd.isna(row['geometry'].x) and not pd.isna(row['geometry'].y and not pd.isna(row['original_magnitude']))
    ]

In [14]:
heatmapdata

[(38.7, 141.1, 7.0),
 (40.3, 43.1, 5.9),
 (11.0, -66.0, 7.7),
 (26.0, 100.1, 6.5),
 (43.4, 28.7, 6.4),
 (40.5, 142.5, 7.2),
 (-43.0, 173.0, 6.8),
 (40.5, 141.3, 6.9),
 (40.7, 48.6, 6.9),
 (40.7, 33.6, 5.5),
 (14.0, -91.0, 7.5),
 (43.2, 129.6, 6.7),
 (40.0, 77.0, 7.7),
 (-35.0, 138.0, 6.0),
 (40.8, 72.3, 6.4),
 (32.2, 35.3, 5.7),
 (39.14, 42.65, 7.0),
 (40.9, 42.7, 5.8),
 (36.36, 22.97, 8.3),
 (35.2, 58.2, 6.5),
 (23.5, 120.5, 6.0),
 (37.75, 27.0, 6.2),
 (30.0, 101.0, 6.8),
 (23.5, 120.3, 6.3),
 (33.0, 76.0, 7.8),
 (42.1, 19.6, 6.6),
 (34.1, 132.5, 7.0),
 (39.0, 16.0, 6.8),
 (40.0, 24.0, 6.8),
 (1.0, -81.5, 8.8),
 (23.6, 120.4, 6.8),
 (23.6, 120.4, 7.1),
 (38.0, -123.0, 7.8),
 (-33.0, -72.0, 8.2),
 (43.5, 85.0, 7.2),
 (2.0, 94.5, 7.5),
 (18.2, -76.7, 6.5),
 (17.0, -100.0, 7.9),
 (14.0, 123.0, 7.1),
 (38.0, 69.0, 7.2),
 (23.0, 121.1, 6.8),
 (38.0, 15.5, 7.2),
 (38.6, 26.9, 5.8),
 (33.0, 50.0, 7.3),
 (40.0, 38.0, 6.8),
 (25.0, 122.5, 7.1),
 (38.9, -8.8, 6.6),
 (24.4, 103.0, 6.5),
 (-2.0, 

In [15]:
hm = HeatMap(data=heatmapdata)

In [16]:
hm.add_to(world_map)

In [17]:
world_map

In [18]:
data_points = []
max_mag = 0
for idx, row in eq_table_geo.sort_values('origin (utc)').iterrows():
    if row['geometry'] and not pd.isna(row['geometry'].x) and not pd.isna(row['geometry'].y):
        data_points.append((row['origin (utc)'].to_pydatetime(), row['geometry'].y, row['geometry'].x, row['original_magnitude']))
        if row['original_magnitude'] > max_mag:
            max_mag = row['original_magnitude']

by_year=defaultdict(list)
for record in data_points:
    by_year[f'{record[0].strftime("%Y")}'].append([record[1],record[2], 1])
    
over_time = []
time_index = []
for key, values in by_year.items():
    over_time.append(values)
    time_index.append(key)

In [19]:
over_time

[[[38.7, 141.1, 1], [40.3, 43.1, 1], [11.0, -66.0, 1]],
 [[26.0, 100.1, 1], [43.4, 28.7, 1], [40.5, 142.5, 1], [-43.0, 173.0, 1]],
 [[40.5, 141.3, 1],
  [40.7, 48.6, 1],
  [40.7, 33.6, 1],
  [14.0, -91.0, 1],
  [43.2, 129.6, 1],
  [40.0, 77.0, 1],
  [-35.0, 138.0, 1],
  [40.8, 72.3, 1]],
 [[32.2, 35.3, 1],
  [39.14, 42.65, 1],
  [40.9, 42.7, 1],
  [36.36, 22.97, 1],
  [35.2, 58.2, 1]],
 [[23.5, 120.5, 1], [37.75, 27.0, 1], [30.0, 101.0, 1], [23.5, 120.3, 1]],
 [[33.0, 76.0, 1],
  [42.1, 19.6, 1],
  [34.1, 132.5, 1],
  [39.0, 16.0, 1],
  [40.0, 24.0, 1]],
 [[1.0, -81.5, 1],
  [23.6, 120.4, 1],
  [23.6, 120.4, 1],
  [38.0, -123.0, 1],
  [-33.0, -72.0, 1],
  [43.5, 85.0, 1]],
 [[2.0, 94.5, 1],
  [18.2, -76.7, 1],
  [17.0, -100.0, 1],
  [14.0, 123.0, 1],
  [38.0, 69.0, 1]],
 [[23.0, 121.1, 1], [38.0, 15.5, 1]],
 [[38.6, 26.9, 1],
  [33.0, 50.0, 1],
  [40.0, 38.0, 1],
  [25.0, 122.5, 1],
  [38.9, -8.8, 1],
  [24.4, 103.0, 1],
  [-2.0, 101.0, 1],
  [43.7, 5.4, 1],
  [35.4, 136.3, 1],
  [30.0

In [20]:
mapheattime = folium.Map(tiles='Stamen Terrain')
hm = HeatMapWithTime(data=over_time,
    index=time_index,
    auto_play=False,
    max_opacity=0.3,
    )
hm.add_to(mapheattime)

In [21]:
mapheattime